In [6]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
from sklearn.metrics import accuracy_score, log_loss
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC, LinearSVC
from sklearn.preprocessing import LabelEncoder
from sklearn.calibration import CalibratedClassifierCV
%matplotlib inline

trd = pd.read_csv('train.csv')
tsd = pd.read_csv('test.csv')
df = pd.concat([trd, tsd], ignore_index=True, sort  = False)

In [7]:
df.shape

(1309, 12)

In [8]:
features= ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [9]:
x = df[features]
y = df['Survived']

In [10]:
x['Age'] = x['Age'].fillna(x['Age'].median())
x['Embarked']= x['Embarked'].fillna(x['Embarked'].value_counts().index[0])

In [18]:
LE = LabelEncoder()
x['Sex'] = LE.fit_transform(x['Sex'])
x['Embarked'] = LE.fit_transform(x['Embarked'])

In [ ]:
test_data = pd.read_csv('../input/titanic/test.csv')
test_x = test_data[features]

In [ ]:
##Now we predict the values
prediction = classifier.predict(test_x)

In [ ]:
td['Sex'] = LabelEncoder().fit_transform(td['Sex'])

In [ ]:
#Создаем фрейм для предсказания
X_to_be_predicted = td[td.Survived.isnull()] #В новом фрейме Удаляем все значения Y в колонке 
X_to_be_predicted = X_to_be_predicted.drop(['Survived'], axis = 1) #Затем удаляем всю колонку целиком

train_data = td #Выбираем фрейм
train_data = train_data.dropna() #В новом фрейме чистим от пустых строк
y = train_data['Survived'] #Зависимая переменная
X = train_data.drop(['Survived'], axis = 1) #Предикторы
train_data.shape 

In [ ]:
#Расщепление на обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42,                                                    
                                                    test_size=0.2 # доля объёма тестового множества
                                                   )

In [ ]:
model = RandomForestClassifier(random_state=42, #зерно датчика случайных чисел
                               n_estimators=100, #число деревьев в лесу
                               criterion='gini', #функция для дельта H, impurity ('gini' или 'entropy')
                               max_depth=5, #Макс число слоев                               
                               oob_score=True, #Вычислять out-of-bag ошибку                               
                               warm_start=False, #использовать результаты предыдущего вызова и нарастить предыдущий лес
                               class_weight=None #веса классов для балансировки выборки для обучения
                              )

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("RF Accuracy: "+repr(round(model.score(X_test, y_test) * 100, 2)) + "%")

result_rf=cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')

print('The cross validated score for Random forest is:',round(result_rf.mean()*100,2))
y_pred = cross_val_predict(model, X_train, y_train, cv=10)

#Важность вклада предикторов
pd.DataFrame({'feature': X.columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

In [ ]:
result = model.predict(X_to_be_predicted)
submission = pd.DataFrame({'PassengerId':X_to_be_predicted.PassengerId,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)
filename = 'Titanic_test_predictions.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

In [ ]:
df = pd.read_csv('Titanic Predictions.csv')